In [1]:
## 1.. Read csv data and load data  ! 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_comnts = pd.read_csv('UScomments.csv',on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: 'UScomments.csv'

In [ ]:
df_comnts.head()

In [ ]:
df_comnts.isnull().sum()

In [ ]:
df_comnts.dropna(inplace=True)

In [ ]:
## 2.. Perform Sentiment Analysis
    

In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob("Logan Paul it's yo big day ‼️‼️‼️").sentiment.polarity

In [ ]:
polarity =[]

try:
    for comments in df_comnts['comment_text']:
        polarity.append(TextBlob(comments).sentiment.polarity)
except:
    polarity.append(0)

df_comnts['polarity'] = polarity
    

In [ ]:
df_comnts.head()

In [ ]:
## 3..  Wordcloud Analysis

In [ ]:
flt1 = (df_comnts['polarity'] == 1)


In [ ]:
comments_positive=df_comnts[flt1]

In [ ]:
comments_positive

In [ ]:
flt2 = df_comnts['polarity']== -1 
comments_negative =df_comnts[flt2]

In [ ]:
comments_negative

In [ ]:
from wordcloud import WordCloud,STOPWORDS

In [ ]:
set(STOPWORDS).head(5)

In [ ]:
WordCloud(stopwords=set(STOPWORDS))

In [ ]:
total_comments_positive =' '.join(comments_positive['comment_text'])

In [ ]:
total_comments_negative =' '.join(comments_negative['comment_text'])

In [ ]:
positive_wordcloud = WordCloud(stopwords=set(STOPWORDS)).generate(total_comments_positive)

In [ ]:
plt.imshow(positive_wordcloud)
plt.show()

In [ ]:
### Conclusion-->> positive Users are emphasizing more on best , awesome , perfect , amazing , look , happy  etc..

In [ ]:
negative_wordcloud = WordCloud(stopwords=set(STOPWORDS)).generate(total_comments_negative)

In [ ]:
plt.imshow(negative_wordcloud)
plt.show()

In [ ]:
### Conclusion-->> Negative Users are emphasizing more on Terrible , worst ,horrible ,boring , disgusting etc..

In [ ]:
## 4.. Emoji's Analysis

In [ ]:
!pip install emoji==2.2.0
import emoji

In [ ]:
emoji_list =[]

for comments in df_comnts['comment_text']:
    for char in comments:
        if char in emoji.EMOJI_DATA:
            emoji_list.append(char)


In [ ]:
from collections import Counter

Counter(emoji_list).most_common(10)

In [ ]:
emojis =[]
for i in range(10):
    emojis.append(Counter(emoji_list).most_common(10)[i][0])

In [ ]:
freq =[]
for i in range(10):
    freq.append(Counter(emoji_list).most_common(10)[i][1])

In [ ]:
!pip install jupyterlab-plotly

import plotly.graph_objs as go
from plotly.offline import iplot


trace = go.Bar(x=emojis,y=freq)
iplot([trace])

In [ ]:
##5.. Collect Entire data of Youtube !

In [ ]:
import os

In [ ]:
files  = os.listdir(r'C:\Users\adiha\OneDrive\Desktop\jupyrt NT\additional_data')

In [ ]:
files

In [ ]:
files_csv =[]
files_json =[]
for file in files:
    if file.endswith('.csv'):
        files_csv.append(file)
    else:
        files_json.append(file)

In [ ]:
files_csv

In [ ]:
files_json

In [ ]:
full_df =pd.DataFrame()

In [ ]:
path = r'C:\Users\adiha\OneDrive\Desktop\jupyrt NT\additional_data'

for files in files_csv:
    current_df = pd.read_csv(path+'/'+files,encoding = 'iso-8859-1')
    
    full_df = pd.concat([full_df,current_df],  ignore_index=True)
    

In [ ]:
full_df.shape

In [ ]:
full_df.duplicated().sum()

In [ ]:
full_df = full_df.drop_duplicates()

In [ ]:
full_df.to_csv(r'C:\Users\adiha\OneDrive\Desktop\jupyrt NT/YouTube_full_df_project.csv',index = False)

In [ ]:
## 7.. Which Category has the maximum likes ?

In [ ]:
json_df =pd.read_json(r'C:\Users\adiha\OneDrive\Desktop\jupyrt NT\additional_data/US_category_id.json')

In [ ]:
json_df

In [ ]:
json_df['items']

In [ ]:
json_df['items'][0]

In [ ]:
category_dict={}

for item in json_df['items'].values:
    category_dict[int(item['id'])] = item['snippet']['title']

category_dict
    

In [ ]:
full_df['category_name']=full_df['category_id'].map(category_dict)

In [ ]:
plt.figure(figsize = (12,8))
sns.boxplot(x='category_name',y='likes',data= full_df)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## 8.. whether audience is engaged or not

In [ ]:
full_df['like_rate']=(full_df['likes']/full_df['views'])*100
full_df['dislike_rate']=(full_df['dislikes']/full_df['views'])*100
full_df['comment_rate']=(full_df['comment_count']/full_df['views'])*100

In [ ]:
plt.figure(figsize = (8,8))
sns.boxplot(x='category_name',y='like_rate',data= full_df)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
sns.regplot(x='views',y='likes',data=full_df)
plt.show()

In [ ]:
full_df[['views','likes','comment_count']].corr()

In [ ]:
sns.heatmap(full_df[['views','likes','comment_count']].corr(),annot=True)

In [ ]:
count_df=full_df['channel_title'].value_counts().reset_index()
count_df = count_df.head(20)

In [ ]:
## 9.. Which channels have the largest number of trending videos?

In [ ]:
sns.barplot(x='count',y='channel_title',data=count_df,palette='coolwarm_r')
plt.xlabel('No. of videos')
plt.show()

In [ ]:
## 10.. Does Punctuations in title and tags have any relation with views, likes, dislikes comments?

In [ ]:
import string
string.punctuation

In [ ]:
punctuation_count=[]

for str in full_df['title']:
    count = 0
    for char in str:
        if char in string.punctuation:
            count += 1
    punctuation_count.append(count)

full_df['No. of punctuations']=punctuation_count

In [ ]:
full_df.head()
full_df.drop(columns='no of punctuation',axis=1)

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='No. of punctuations',y='views',data=full_df)
plt.show()